# How to Start With a Raw Dataset?

This demo shows how to deal with the raw datasets with reviews into lexicon format, which is required by sentiment-related models, e.g. EFM, MTER...

### Step 1: Prepare the raw data file
**Make sure the first line in the file indicates the column names**

At least including 4 columns named [user_id, item_id, rating, review_text], see example ./dataset/data_demo.csv


In [66]:
raw_file = './dataset/data_demo.csv'
sep = '\t'

### Step 2: Generate Lexicon
**Use the ```cornac.data.lexicon.SentimentAnalysis```**

- Input: raw data file path;
- Output: 
  - rating.txt: [user_id, item_id, rating]
  - lexicon.txt: [user_id, item_id, lexicon]

*Note: The user-item pairs in the two ouput files are consistent with each other.*

In [67]:
import warnings
warnings.filterwarnings('ignore') 
from cornac.data.lexicon import SentimentAnalysis

In [68]:
output_lexicon = './dataset/lexicon.txt'
output_rating = './dataset/rating.txt'
# The parameter usecols is required, should be the same with the first line of the raw file
SA = SentimentAnalysis(raw_file, sep=sep, usecols = ['user_id', 'item_id', 'rating', 'review_text']) 
df = SA.build_lexicons()
SA.save_to_file(output_lexicon, output_rating)
df.head()

  0%|          | 0/60 [00:00<?, ?it/s]

100%|██████████| 60/60 [00:00<00:00, 101.53it/s]

number of users: 12
number of items: 10
total60
15 rows have no lexicon
45 rows after dropping users having less than 1 reviews


,user_id,item_id,rating,review_text,lexicon
0,713cc3505e77532b97d0a69812320fa7,4303163,2.0,"Complex and captivating, DARKLY showcases Pess...","twists:unexpected:1,page:final:1,delight:absol..."
1,5f0d7ea4515a98abebea35cec77f864c,192805,4.0,I was absolutely captivated by the synopsis fo...,"thing:kinda:1,job:amazing:1"
5,9003d274774f4c47e62f77600b08ac1d,23167683,3.0,A failure on nearly every level. The character...,"voice:preposterous:1,concern:main:1,stumps:pro..."
6,9131e02af6b7d8d2dd23472b264971af,23167683,4.0,I believe this book will be in my top 3 I have...,duties:everyday:1
7,ba2455719e99ae6e0771877da9e81474,48100,3.0,An interesting twist on the black ops/espionag...,"twist:interesting:1,genres:black:1,point:polit..."


### Step 3 Train Recommendation Models

In [69]:

from cornac.data import Reader
from cornac.experiment.experiment import Experiment
from cornac.models import MTER, EFM
from cornac.eval_methods import RatioSplit
from cornac.data import SentimentModality

In [70]:
reader = Reader()
ratings = reader.read(output_rating, fmt='UIR', sep=',')
lexicon = reader.read(output_lexicon, fmt='UITup', sep=',', tup_sep=':')
sentiment = SentimentModality(data = lexicon)

In [71]:
rs = RatioSplit(data=ratings, 
                sentiment = sentiment,
                test_size=0.2, 
                rating_threshold=4.0, 
                seed=123)

In [72]:
efm = EFM()
efm.fit(rs.train_set)

In [73]:
efm.recommend("713cc3505e77532b97d0a69812320fa7")

['342923',
 '48100',
 '192805',
 '23167683',
 '1171422',
 '4303163',
 '27423576',
 '7932435']